In [1]:
import katagames_sdk.engine as kataen
from scripts import *
pygame = kataen.import_pygame()
EventReceiver = kataen.EventReceiver
EngineEvTypes = kataen.EngineEvTypes
scr_size = None


class player():
    def __init__(self, id_name_colour):
        self.idx = id_name_colour[0]
        self.name = id_name_colour[1]
        self.colour = id_name_colour[2]
        self.score = 0
        self.tally = []
        
    def reset(self):
        self.score = 0
        self.tally = []
        
        
def list_columns(obj, rows=12, gap=2):
    '''
    adapted from https://stackoverflow.com/questions/1524126/how-to-print-a-list-more-nicely/25048690
    '''
    gap = 2 if len(obj)>rows else 0
    
    if len(obj)>0:
        sobj = [str(item) for item in obj]
        max_len = max([len(item) for item in sobj])

        plist = [sobj[i: i+rows] for i in range(0, len(sobj), rows)]
        if not len(plist[-1]) == rows:
            plist[-1].extend(['']*(len(sobj) - len(plist[-1])))
        plist = zip(*plist)
        printer = [''.join([c.rjust(max_len + gap) for c in p]) for p in plist]
        printer = [x[gap:] for x in printer]
    else:
        printer = ['']
    
    return printer

class TicTacHoe(EventReceiver):
    
    def __init__(self, nrows, ncols, width=20, margin=4):
        self.caption = 'Tic-Tac-Hoe'
        self.nrows = nrows
        self.ncols = ncols
        self.width = width
        self.margin = margin//2*2
        self.full_width = self.width + self.margin
        self.size = (self.ncols*self.full_width + self.margin + 300, self.nrows*self.full_width + self.margin)
        
        self.colours = {
            'BLACK': (0, 0, 0),
            'GREY': (220, 220, 220),
            'WHITE': (255, 255, 255),
            'RED': (255, 0, 0),
            'GREEN': (0, 255, 0),
            'BLUE': (0, 0, 255)
        }
        
        self.reset_tables()
        self.selection = ()

    def reset_tables(self):
        self.grid = np.zeros([self.nrows, self.ncols], dtype=int)
        self.rows = np.ones([self.nrows-1, self.ncols], dtype=int)*7
        self.cols = np.ones([self.nrows, self.ncols-1], dtype=int)*7
        
    def create_players(self, p1, p2):
        self.p1 = player(p1)
        self.p2 = player(p2)
        self.players = {1: self.p2, 2: self.p1}
        self.active_player = self.p1
    
    def update_scores(self, player):
        mgrid = merge(self.grid, self.rows, self.cols)
        ccs = get_cc(mgrid)
        lengths = np.array([len(x) for x in ccs])
        maks = np.argmax(lengths)
        idx = list(range(len(lengths)))[:maks] + list(range(len(lengths)))[maks+1:]
        score = 0
        for i in idx:
            for j in ccs[i]:
                mgrid[j[0], j[1]] = player.idx
                score += 1
                
        if score > 0:
            player.score += score
            player.tally.append(score)
            print(list_columns(player.tally))
        self.grid, self.rows, self.cols = split(mgrid)
        
    def draw_line(self, player):
        if [len(x) for x in self.selection]==[1, 3]:
            self.rows[self.selection] = 8
            self.active_player = self.players[player.idx]
            
            self.update_scores(player)
            
        elif [len(x) for x in self.selection]==[3, 1]:
            self.cols[self.selection] = 8
            self.active_player = self.players[player.idx]
            
            self.update_scores(player)
        
    def draw_board(self, display, mouse, event):
        
        tile_colour = self.colours['WHITE']
        border_colour = self.colours['BLACK']
        ht_colour = self.colours['GREEN']
        display.fill(self.colours['BLACK'])
        pygame.draw.rect(display, self.colours['GREY'],
                                 [self.margin, self.margin,
                                  self.full_width*self.ncols - self.margin,
                                  self.full_width*self.nrows - self.margin])
        pygame.draw.rect(display, self.colours['GREY'],
                                 [self.full_width*self.ncols + self.margin*2,
                                  self.margin, 300 - + self.margin*3,
                                  self.full_width*self.nrows - self.margin])
        pygame.draw.rect(display, self.colours['BLACK'],
                                 [self.full_width*self.ncols + self.margin*2,
                                  self.margin*7, 300 - + self.margin*3,
                                  self.margin/2])
        pygame.draw.rect(display, self.colours['BLACK'],
                                 [self.full_width*self.ncols + self.margin*2,
                                  self.margin*11, 300 - + self.margin*3,
                                  self.margin/2])
        
        
        # Draw scores
        font36 = pygame.font.SysFont('dejavusansmono', 36)
        font24 = pygame.font.SysFont('dejavusansmono', 24)
        star = font36.render('*', True, self.active_player.colour)
        p1 = font36.render(self.p1.name, True, self.p1.colour)
        p2 = font36.render(self.p2.name, True, self.p2.colour)
        s1 = font36.render(str(self.p1.score), True, self.p1.colour)
        s2 = font36.render(str(self.p2.score), True, self.p2.colour)
        
        display.blit(star, dest=(self.full_width*self.ncols + self.margin*2 + 67.5 - star.get_rect().width / 2 + (self.active_player.idx - 1) * 135, self.margin*0.9))
        display.blit(p1, dest=(self.full_width*self.ncols + self.margin*2 + 67.5 - p1.get_rect().width / 2, self.margin*2.7))
        display.blit(p2, dest=(self.full_width*self.ncols + self.margin*2 + 67.5 + 135 - p2.get_rect().width / 2, self.margin*2.7))
        display.blit(s1, dest=(self.full_width*self.ncols + self.margin*2 + 67.5 - s1.get_rect().width / 2, self.margin*7.3))
        display.blit(s2, dest=(self.full_width*self.ncols + self.margin*2 + 67.5 + 135 - s2.get_rect().width / 2, self.margin*7.3))
        
        offset = 0
        for i in list_columns(self.p1.tally):
            t1 = font24.render(i, True, self.p1.colour)
            display.blit(t1, dest=(self.full_width*self.ncols + self.margin*2 + 67.5 - t1.get_rect().width / 2, self.margin*12 + offset))
            offset += 30
        
        offset = 0
        for i in list_columns(self.p2.tally):
            t2 = font24.render(i, True, self.p2.colour)
            display.blit(t2, dest=(self.full_width*self.ncols + self.margin*2 + 67.5 + 135 - t2.get_rect().width / 2, self.margin*12 + offset))
            offset += 30
        

        

        
        # Draw the grid
        for row in range(self.nrows):
            for col in range(self.ncols):
                value = self.grid[row, col]
                colour = [tile_colour, self.p1.colour, self.p2.colour][value]
                pygame.draw.rect(display, colour,
                                 [self.full_width*col + self.margin,
                                  self.full_width*row + self.margin,
                                  self.width, self.width])

        # Draw the row borders
        row_borders = np.array(np.where(self.rows==8)).T
        for i in row_borders:
            pygame.draw.rect(display, border_colour,
                             [self.full_width*i[1] + self.margin/2,
                              self.full_width*i[0] + self.full_width,
                              self.full_width, self.margin])

        # Draw the col borders
        col_borders = np.array(np.where(self.cols==8)).T
        for i in col_borders:
            pygame.draw.rect(display, border_colour,
                             [self.full_width*i[1] + self.full_width,
                              self.full_width*i[0] + self.margin/2,
                              self.margin, self.full_width])
            
        # Highlight selection
        c = (mouse[0] - self.margin/2) / self.full_width
        r = (mouse[1] - self.margin/2) / self.full_width
        
        self.r = r
        self.c = c
        self.selection = ()

        distancetocol = np.abs(np.round(c) - c)
        distancetorow = np.abs(np.round(r) - r)
        if distancetorow < distancetocol:
            rr = int(np.round(r))
            rc = int(np.round(c - 1.5))
            rect = [0, 0, 0, 0]
            if rr > 0 and rr < self.nrows:
                if len(self.rows[rr-1, rc:rc+3]) == 3:
                    if (self.rows[rr-1, rc:rc+3] == np.array([7, 7, 7])).all():
                        rect = [rc*self.full_width,
                                rr*self.full_width,
                                self.margin + 3*self.full_width, self.margin]     
                        self.selection = ([rr-1], list(range(rc, rc+3)))

        else:
            rr = int(np.round(r - 1.5))
            rc = int(np.round(c))
            rect = [0, 0, 0, 0]
            if rc > 0 and rc < self.ncols:
                if len(self.cols[rr:rr+3, rc-1]) == 3:
                    if (self.cols[rr:rr+3, rc-1] == np.array([7, 7, 7])).all():
                        rect = [np.round(c)*self.full_width,
                                np.round(r - 1.5)*self.full_width,
                                self.margin, self.margin + 3*self.full_width]
                        self.selection = (list(range(rr, rr+3)), [rc-1])

        pygame.draw.rect(display, ht_colour, rect)

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
[katasdk 0.0.6 - https://kata.games/developers]


In [9]:
pygame.init()
TTH = TicTacHoe(10, 10, width=40, margin=10)
TTH.create_players((1, 'p1', 'RED'), (2, 'p2', 'BLUE'))
pygame.display.set_caption(TTH.caption)
display = pygame.display.set_mode(TTH.size)

# TTH.rows[3, 5:7] = 8
# TTH.cols[4, 6:8] = 8

while True:
    for event in pygame.event.get():
        if event.type == pygame.QUIT:
            pygame.quit()
            sys.exit()
            
        if event.type == pygame.MOUSEBUTTONDOWN:
            TTH.draw_line(TTH.active_player)

            
            print(pygame.mouse.get_pos())
            print(TTH.r, TTH.c)
        

    # Draw the tilemap
    TTH.draw_board(display, pygame.mouse.get_pos(), event)

    pygame.display.update()

(227, 155)
3.0 4.44
(300, 213)
4.16 5.9
(228, 227)
4.44 4.46
['3']
(163, 187)
3.64 3.16


SystemExit: 

In [1]:
import pygame
from pygame.locals import QUIT, KEYDOWN, KEYUP, K_UP, K_DOWN
scr_size = (480, 270)
avpos = [240, 135]
avdir = 0
gameover = False
clock = pygame.time.Clock()
pygame.init()
screen = pygame.display.set_mode(scr_size)
while not gameover:
    for ev in pygame.event.get():
        if ev.type == QUIT:
            gameover = True
        elif ev.type == KEYDOWN:
            if ev.key == K_UP:
                avdir = -1
            elif ev.key == K_DOWN:
                avdir = 1
        elif ev.type == KEYUP:
            prkeys = pygame.key.get_pressed()
            if not(prkeys[pygame.K_UP] or prkeys[pygame.K_DOWN]):
                avdir = 0
    # update logic, draw
    avpos[1] = (avpos[1] + avdir) % scr_size[1]
    screen.fill(pygame.color.Color('antiquewhite2'))
    pygame.draw.circle(screen, (244,105,251), avpos, 15, 0)
    pygame.display.flip()
    clock.tick(60)
pygame.quit()
print('game over.')

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
game over.


In [1]:
import katagames_sdk.engine as kataen
pygame = kataen.import_pygame()
EventReceiver = kataen.EventReceiver
EngineEvTypes = kataen.EngineEvTypes
scr_size=None

class Avatar:
    def __init__(self):
        self.pos = [240, 135]
        self.direct = 0

class AvatarView(EventReceiver):
    def __init__(self, avref):
        super().__init__()
        self.avref = avref
    def proc_event(self, ev, source):
        if ev.type == EngineEvTypes.PAINT:
            ev.screen.fill(pygame.color.Color('antiquewhite2'))
            pygame.draw.circle(ev.screen, (244,105,251), self.avref.pos, 15, 0)

class AvatarCtrl(EventReceiver):
    def __init__(self, avref):
        super().__init__()
        self.avref = avref
    def proc_event(self, ev, source):
        global scr_size
        if ev.type == EngineEvTypes.LOGICUPDATE:
            avdir = self.avref.direct
            self.avref.pos[1] = (self.avref.pos[1] + avdir) % scr_size[1]
        elif ev.type == pygame.KEYDOWN:
            if ev.key == pygame.K_UP:
                self.avref.direct = -1
            elif ev.key == pygame.K_DOWN:
                self.avref.direct = 1
        elif ev.type == pygame.KEYUP:
            prkeys = pygame.key.get_pressed()
            if not(prkeys[pygame.K_UP] or prkeys[pygame.K_DOWN]):
                self.avref.direct = 0

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html
[katasdk 0.0.6 - https://kata.games/developers]


In [2]:
def run_game():
    global scr_size
    kataen.init(kataen.HD_MODE)
    scr_size = kataen.get_screen().get_size()
    av = Avatar()
    li_recv = [kataen.get_game_ctrl(), AvatarView(av), AvatarCtrl(av)]
    for recv_obj in li_recv:
        recv_obj.turn_on()
    li_recv[0].loop()
    kataen.cleanup()

if __name__=='__main__':
    run_game()

<->context: genuine Pygame
clean exit: OK
